# Preprocesamiento para Experimentos de Actividad Evocada

Este Cuaderno es básicamente una versión interactiva del programa en el archivo "SeparaActividadyPromedia.jl" que contiene las mismas rutinas pero automatizadas. Esta versión permite un poco más de interacción.

In [1]:
push!(LOAD_PATH, ".")
# en este modulo se encuentran las rutinas para encontrar los estimulos ("trancazos")
# y los canales saturados
using SeparaActividadySaturados01
using HDF5

In [2]:
# Nombre de un archivo *brw en formato hdf5 con datos de acticidad evocada
abrestring="/home/karel/RafaelGutierrez/BRWFiles/050815_3R7_converted.brw"
stringgeneral=replace(abrestring, ".brw"=>"")  

"/home/karel/RafaelGutierrez/BRWFiles/050815_3R7_converted"

In [3]:
#Todos los datos de la matriz del brw file
Datos=AbreyCheca(abrestring)

Dict{String,Any} with 8 entries:
  "minvolt"     => -4125.0
  "maxvolt"     => 4125.0
  "frecuencia"  => 7020.8
  "DatosCrudos" => UInt16[0x0000 0x0000 … 0x0000 0x0000; 0x0800 0x0809 … 0x07ed…
  "bitdepth"    => 0x0c
  "factor"      => 2.01416
  "numcuadros"  => 210624
  "duracion"    => 30.0

In [4]:
# Si los datos andan en un arreglo de lista en lugar de cuadrado, los ponemos cuadrados
if size(Datos["DatosCrudos"])[1] != 4096
DatosCrudosArreglados=reshape(Datos["DatosCrudos"], (4096, Datos["numcuadros"]))
else
DatosCrudosArreglados=Datos["DatosCrudos"]
end;


In [5]:
#parametetros globales.
freq=Datos["frecuencia"]/1000 #cuadros por milisegundo.
factor=Datos["factor"] #Factor de conversion de numeros enteros a microVolts

retrazo=round(Int, ceil(5*freq))
final=round(Int, ceil(60*freq))
latencia=round(Int, ceil(1.5*freq))
#Gis recomienda esperar hasta 6.5 ms por cada estimulo.
tiempopostgolpe=round(Int,ceil(6.5*freq))


46

Tomamos un electrodo como ejemplo y buscamos los momentos (en cuadros, unidades enteras) de los estimulos:

In [6]:
ejemplo=vec(DatosCrudosArreglados[1500,:]);
listongas=EncuentraTrancazosRaw(ejemplo,500)
println("Sitios de trancazo electrico en cuadros =", listongas)

Sitios de trancazo electrico en cuadros =[47149, 117758, 188366]


Ahora creamos un diccionario que contenga la actividad alrededor de los estimulos

In [7]:
ActividadRaw=ActivAlrededorTrancazo(listongas, DatosCrudosArreglados, freq)
(bla,tmax)=size(ActividadRaw["Trancazo_1"])
println("Tiempo, en cuadros, de los intervalos con trancazo=",tmax)


Tiempo, en cuadros, de los intervalos con trancazo=459


Finalmente, centramos y  reescalamos los datos, y los promediamos.

In [8]:
DatosCentrados=Dict{AbstractString, Array}()
for k in keys(ActividadRaw)
    DatosCentrados[k]=FormaMatrizDatosCentrados(ActividadRaw[k], factor)
end
LFPPromedio=(DatosCentrados["Trancazo_1"]+DatosCentrados["Trancazo_2"]+DatosCentrados["Trancazo_3"])/3;

Eso fue la parte de promediar y poner escala sensible a los datos. Ahora prosigue detectar canales con respuesta fisiológica y canales saturados. Empecemos primero por las respuestas.

In [9]:
#los parametros son datos promediados, frecuencia, tiempo post estimulo en ms, umbral en microvolts, umbral de saturación en microvolts
PruebaRespuesta=BuscaCanalRespActPot(LFPPromedio,freq, 120,-120,-800)
# y luego los saturados
iniciobusqueda=retrazo
finbusqueda=retrazo+4*latencia
Saturados=BuscaSaturados(LFPPromedio,1200,iniciobusqueda,finbusqueda)
numerosaturados=length(Saturados)
println("Encontramos ", numerosaturados, " canales probablemente saturados.")


Encontramos 274 canales probablemente saturados.


In [10]:
dirname(stringgeneral), basename(stringgeneral)

("/home/karel/RafaelGutierrez/BRWFiles", "050815_3R7_converted")

Finalmente podemos guardar los Datos

In [12]:
dirgen=dirname(stringgeneral)
basegen=basename(stringgeneral)
outname=string(dirgen,"/LFP_Promedio_", basegen, ".h5")
#aqui hay que arreglar el rollo del directorio
parametros=(freq, retrazo, latencia,final)
listaaguardar=Dict(
     "LFPSaturados"=>LFPPromedio,
     "freq" =>freq,
     "freq" => freq,
     "retrazo" => retrazo,
     "latencia" =>latencia,
     "final"=> final,
     "Canalesrespuesta"=>PruebaRespuesta,
     "CanalesSaturados" => Saturados)


println(outname )
   h5open(outname, "w")  do file
        for k in keys(listaaguardar)
            datos=listaaguardar[k]
            println("abriendo los datos ", k)
            println
            write(file, "$k", datos)
        end
    end

/home/karel/RafaelGutierrez/BRWFiles/LFP_Promedio_050815_3R7_converted.h5
abriendo los datos Canalesrespuesta
abriendo los datos CanalesSaturados
abriendo los datos latencia
abriendo los datos retrazo
abriendo los datos LFPSaturados
abriendo los datos freq
abriendo los datos final


In [14]:
outname

"/home/karel/RafaelGutierrez/BRWFiles/LFP_Promedio_050815_3R7_converted.h5"